# Collecting Football Datasets with Pandas
In this notebook, we web scrape a football dataset from fbref.com.

## 1. Example of scraping a table using Pandas

### 1.1 Read in the table

Firstly, we load the libraries required for reading the tables from the web pages and storing them in our local computer.

In [1]:
import requests
import pandas as pd
import os
from pathlib import Path

In [2]:
# disable pandas warnings
import warnings
warnings.filterwarnings('ignore')

We specify the URL to the table, and read the table using the `read_html` method in the pandas library. In this example, we use the *id* tag to identify the table to be read from the web page.

In [2]:
URL = "https://fbref.com/en/comps/Big5/2020-2021/stats/players/"

In [117]:
stats_standard_lst = pd.read_html(URL, attrs = {'id': 'stats_standard'},  flavor='bs4')
stats_standard_df = stats_standard_lst[0]
stats_standard_df.head() # output first 5 rows

Unnamed: 0_level_0   Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk               Player             Nation   
0                  1        Ismael Aaneba             fr FRA   
1                  2  Patrick van Aanholt             nl NED   
2                  3          Issah Abbas             gh GHA   
3                  4     Yunis Abdelhamid             ma MAR   
4                  5        Sabit Abdulai             gh GHA   

  Unnamed: 3_level_0 Unnamed: 4_level_0  Unnamed: 5_level_0  \
                 Pos              Squad                Comp   
0              FW,DF         Strasbourg          fr Ligue 1   
1                 DF     Crystal Palace  eng Premier League   
2              DF,FW           Mainz 05       de Bundesliga   
3                 DF              Reims          fr Ligue 1   
4                 MF             Getafe          es La Liga   

  Unnamed: 6_level_0 Unnamed: 7_level_0 Playing Time         ... Expected  \
                 Age               Born           MP Starts  ...       xG   
0                 21               1999            2      0  ...      0.0   
1                 29               1990           22     20  ...      1.2   
2                 21               1998            2      0  ...      0.0   
3                 32               1987           33     33  ...      1.7   
4                 21               1999            3      0  ...      0.0   

                    Per 90 Minutes                            \
  npxG   xA npxG+xA             xG    xA xG+xA  npxG npxG+xA   
0  0.0  0.0     0.0           0.00  0.00  0.00  0.00    0.00   
1  1.2  0.8     2.0           0.06  0.04  0.10  0.06    0.10   
2  0.0  0.0     0.0           0.00  0.00  0.00  0.00    0.00   
3  1.7  0.3     1.9           0.05  0.01  0.06  0.05    0.06   
4  0.0  0.0     0.0           0.00  0.00  0.00  0.00    0.00   

  Unnamed: 33_level_0  
              Matches  
0             Matches  
1             Matches  
2             Matches  
3             Matches  
4             Matches  

[5 rows x 34 columns]

### 1.2 Clean the data

We perform data cleaning to handle the issues below:
- drop repeated column headers embedded within the table
- Flatten the multi-index column
- remove the first element in the **Nation** column (e.g. convert "fr FRA" to "FRA")
- keep only the first position of each player in the comma separated string in the **Pos** column
- remove the first element in the **Comp** column (e.g. convert "fr Ligue 1" to "Ligue 1")
- drop unnecessary columns - **Rk** and **Matches**

### 1.2.1 Drop repeated column headers

In [118]:
stats_standard_df.iloc[25:30] # repeated column header can be seen in row 25

Unnamed: 0_level_0    Unnamed: 1_level_0 Unnamed: 2_level_0  \
                   Rk                Player             Nation   
25                 Rk                Player             Nation   
26                 26  Martin Agirregabiria             es ESP   
27                 27         Lucien Agoume             fr FRA   
28                 28             Felix Agu             de GER   
29                 29         Kevin Agudelo             co COL   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0  \
                  Pos              Squad               Comp   
25                Pos              Squad               Comp   
26              DF,MF             Alavés         es La Liga   
27                 MF             Spezia         it Serie A   
28              DF,FW      Werder Bremen      de Bundesliga   
29                 FW             Spezia         it Serie A   

   Unnamed: 6_level_0 Unnamed: 7_level_0 Playing Time          ... Expected  \
                  Age               Born           MP  Starts  ...       xG   
25                Age               Born           MP  Starts  ...       xG   
26                 24               1996           26      16  ...      0.2   
27                 18               2002           12       9  ...      0.1   
28                 20               1999           15       8  ...      0.6   
29                 21               1998           29      14  ...      1.9   

                       Per 90 Minutes                              \
    npxG   xA  npxG+xA             xG    xA  xG+xA  npxG  npxG+xA   
25  npxG   xA  npxG+xA             xG    xA  xG+xA  npxG  npxG+xA   
26   0.2  0.9      1.1           0.01  0.05   0.06  0.01     0.06   
27   0.1  0.1      0.1           0.01  0.01   0.02  0.01     0.02   
28   0.6  0.4      1.0           0.07  0.05   0.13  0.07     0.13   
29   1.9  0.7      2.6           0.13  0.05   0.17  0.13     0.17   

   Unnamed: 33_level_0  
               Matches  
25             Matches  
26             Matches  
27             Matches  
28             Matches  
29             Matches  

[5 rows x 34 columns]

In [119]:
stats_standard_df = stats_standard_df.drop_duplicates(keep=False) # remove column headers embedded in table rows


### 1.2.2 Flatten multi-index column

In [120]:
# view level 0 in multi-index
stats_standard_df.columns.get_level_values(0)

Index(['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0',
       'Unnamed: 3_level_0', 'Unnamed: 4_level_0', 'Unnamed: 5_level_0',
       'Unnamed: 6_level_0', 'Unnamed: 7_level_0', 'Playing Time',
       'Playing Time', 'Playing Time', 'Playing Time', 'Performance',
       'Performance', 'Performance', 'Performance', 'Performance',
       'Performance', 'Performance', 'Per 90 Minutes', 'Per 90 Minutes',
       'Per 90 Minutes', 'Per 90 Minutes', 'Per 90 Minutes', 'Expected',
       'Expected', 'Expected', 'Expected', 'Per 90 Minutes', 'Per 90 Minutes',
       'Per 90 Minutes', 'Per 90 Minutes', 'Per 90 Minutes',
       'Unnamed: 33_level_0'],
      dtype='object')

In [121]:
# view level 1 in multi-index
stats_standard_df.columns.get_level_values(1)

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', 'MP',
       'Starts', 'Min', '90s', 'Gls', 'Ast', 'G-PK', 'PK', 'PKatt', 'CrdY',
       'CrdR', 'Gls', 'Ast', 'G+A', 'G-PK', 'G+A-PK', 'xG', 'npxG', 'xA',
       'npxG+xA', 'xG', 'xA', 'xG+xA', 'npxG', 'npxG+xA', 'Matches'],
      dtype='object')

Get valid list of strings for level 0 multi-index

In [122]:
# fill unnamed columns with last valid column name value and replace nulls to empty string
temp = stats_standard_df.columns.get_level_values(0).to_series().mask(lambda x: x.str.startswith('Unnamed')).ffill()
print(temp[:10])

Unnamed: 0_level_0             NaN
Unnamed: 1_level_0             NaN
Unnamed: 2_level_0             NaN
Unnamed: 3_level_0             NaN
Unnamed: 4_level_0             NaN
Unnamed: 5_level_0             NaN
Unnamed: 6_level_0             NaN
Unnamed: 7_level_0             NaN
Playing Time          Playing Time
Playing Time          Playing Time
dtype: object


In [123]:
temp = temp.reset_index(drop=True) # drop index 
temp = temp.fillna("") # replace NaN with empty string
print(temp[:10])

0                
1                
2                
3                
4                
5                
6                
7                
8    Playing Time
9    Playing Time
dtype: object


In [124]:
# concatenate values from level 0 and level 1 and strip whitespace
temp = temp + ' ' + stats_standard_df.columns.get_level_values(1)
temp = temp.apply(lambda x: x.strip())
print(temp[:10])

0                     Rk
1                 Player
2                 Nation
3                    Pos
4                  Squad
5                   Comp
6                    Age
7                   Born
8        Playing Time MP
9    Playing Time Starts
dtype: object


In [125]:
stats_standard_df.columns = temp

### 1.2.3 Clean data in select columns 
We process the strings in Nation, Pos, Comp. Then, we drop the unnecessary column Rk and Matches which are the first and last columns.

In [126]:
# check for null values
stats_standard_df[stats_standard_df.isnull().any(axis=1)]

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,Playing Time MP,Playing Time Starts,...,Expected xG,Expected npxG,Expected xA,Expected npxG+xA,Per 90 Minutes xG,Per 90 Minutes xA,Per 90 Minutes xG+xA,Per 90 Minutes npxG,Per 90 Minutes npxG+xA,Per 90 Minutes Matches
2484,2390,Malik Sellouki,NaN,"FW,MF",Nice,fr Ligue 1,20,2000,4,0,...,0.3,0.3,0.0,0.3,0.35,0.00,0.35,0.35,0.35,Matches


To keep the rows with  null values, we have to manually find and set the missing data. For this example, we will just use set missing values to "NULL" for simplicity

In [127]:
stats_standard_df['Nation'] = stats_standard_df['Nation'].fillna("NULL")
stats_standard_df['Pos'] = stats_standard_df['Pos'].fillna("NULL")
stats_standard_df['Comp'] = stats_standard_df['Comp'].fillna("NULL")

In [128]:
stats_standard_df['Nation'] = stats_standard_df['Nation'].apply(lambda x: x.split()[-1])
stats_standard_df['Pos'] = stats_standard_df['Pos'].apply(lambda x: x.split(',')[0])
stats_standard_df['Comp'] = stats_standard_df['Comp'].apply(lambda x: ' '.join(x.split()[1:]))
stats_standard_df = stats_standard_df.drop(stats_standard_df.columns[[0, len(stats_standard_df.columns)-1]], axis=1) # drop Rk and Matches columns
stats_standard_df.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,Playing Time MP,Playing Time Starts,Playing Time Min,...,Per 90 Minutes G+A-PK,Expected xG,Expected npxG,Expected xA,Expected npxG+xA,Per 90 Minutes xG,Per 90 Minutes xA,Per 90 Minutes xG+xA,Per 90 Minutes npxG,Per 90 Minutes npxG+xA
0,Ismael Aaneba,FRA,FW,Strasbourg,Ligue 1,21,1999,2,0,11,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
1,Patrick van Aanholt,NED,DF,Crystal Palace,Premier League,29,1990,22,20,1777,...,0.05,1.2,1.2,0.8,2.0,0.06,0.04,0.10,0.06,0.10
2,Issah Abbas,GHA,DF,Mainz 05,Bundesliga,21,1998,2,0,18,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
3,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,32,1987,33,33,2889,...,0.09,1.7,1.7,0.3,1.9,0.05,0.01,0.06,0.05,0.06
4,Sabit Abdulai,GHA,MF,Getafe,La Liga,21,1999,3,0,60,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00


Example URLs from FB Ref
1. Big 5 leagues goalkeeping stats: [https://fbref.com/en/comps/Big5/2020-2021/keepers/players/]()
2. Big 5 league shooting stats: [https://fbref.com/en/comps/Big5/2020-2021/shooting/players/]()
3. EPL goalkeeping stats: [https://fbref.com/en/comps/9/2021-2022/keepers/]()

## 2. Define Utility Functions for Reading Tables from FBRef Website

In [12]:
def url_builder(comp_name, season, section):
    """ Return FBRef URL based on competition name
    """
    url = ""
    scheme_domain_language_str = "https://fbref.com/en"
    url = f"{scheme_domain_language_str}/comps/{comp_name}/{season}/{section}/"
    if str.lower(comp_name) == "big5":
        url += "players/"
    return url

In [13]:
def flatten_multiindex_columns(df):
    top_level_new = df.columns.get_level_values(0).to_series().mask(lambda x: x.str.startswith('Unnamed')).ffill()
    top_level_new = top_level_new.reset_index(drop=True)
    top_level_new = top_level_new.fillna("")
    flat_index = top_level_new + " " + df.columns.get_level_values(1)
    flat_index = flat_index.apply(lambda x:x.strip())
    return flat_index

In [14]:
def log_missing_rows(df, curr_fname, log_fname='missing.csv'):
    outdir = '../' + "logs"
    if not os.path.exists(outdir):
        Path(outdir).mkdir(parents=True)
    fullname = os.path.join(outdir, log_fname)

    missing_rows_idx_lst = df[df.isnull().any(axis=1)].index.tolist()
    if len(missing_rows_idx_lst) > 0:
        missing_rows_str = ';'.join(list(map(str, missing_rows_idx_lst)))
        temp = pd.DataFrame({"file":[curr_fname], "missing_rows":[missing_rows_str]})
        temp.to_csv(fullname, index=False, mode='a', header=not os.path.exists(fullname))

In [15]:
def get_cleaned_cols(df):
    # temporarily replace NAs with null string
    df['Nation'] = df['Nation'].fillna("NULL")
    df['Pos'] = df['Pos'].fillna("NULL")
    df['Comp'] = df['Comp'].fillna("NULL")
    
    df['Nation'] = df['Nation'].apply(lambda x: x.split()[-1])
    df['Pos'] = df['Pos'].apply(lambda x: x.split(',')[0])
    df['Comp'] = df['Comp'].apply(lambda x: ' '.join(x.split()[1:]))
    df = df.drop(df.columns[[0, len(df.columns)-1]], axis=1) # drop Rk and Matches columns
    df = df.replace("NULL", pd.NA)
    return df

In [16]:
def get_table_from_url(url_str, table_id):
    results = pd.read_html(url_str, attrs = {'id': table_id},  flavor='bs4')
    df = results[0] # retrieve first table in results list
    df = df.drop_duplicates(keep=False) # remove "column headers" embedded in table rows
    df = df.reset_index(drop=True)
    df.columns = flatten_multiindex_columns(df)
    df = get_cleaned_cols(df)
    return df

In [17]:
def save_table(df, dirname, filename):
    outname = filename

    outdir = '../' + dirname
    if not os.path.exists(outdir):
        Path(outdir).mkdir(parents=True)
    fullname = os.path.join(outdir, outname)
    log_missing_rows(df=df, curr_fname=str(fullname))
    df.to_csv(fullname, index=False)

In [18]:
def collect_tables_in_period(comp_name, period, section_table_lookup):
    """ Collect player data from given competition over the specified period.

    Args:
        comp_name (str): The name of the competition.
        period (int, int): A tuple of two integers specifying the start year and end year. 
            Season year should be the year the season ends in (e.g. For 2021/2022 season, specify year as 2022).
    """
    seasons_lst = [str(i-1) + '-' + str(i) for i in range(period[0], period[1]+1)]

    for season in seasons_lst:
        for section, table_id in section_table_lookup.items():
            print(f"Collecting {section} data for competition:{comp_name}, season:{season}")
            try:
                curr_url = url_builder(comp_name, season, section)
                curr_table = get_table_from_url(curr_url, table_id)
                curr_save_dir = f"datasets/{comp_name}/{season}"
                curr_fname = section + ".csv"
                save_table(curr_table, curr_save_dir, curr_fname)
                print(f"\tData saved to {curr_save_dir}/{curr_fname}")
            except Exception as e:
                print(f"\t{e}")
    print("Done")

## 3. Scrape tables from web page
Lastly, we collect data from the big 5 european football leagues from 2017/2018 to 2021/2022.

In [19]:
section_table_lookup = {
    'stats':'stats_standard',
    'keepers': 'stats_keeper',
    'keepersadv':'stats_keeper_adv',
    'shooting':'stats_shooting',
    'passing':'stats_passing',
    'passing_types':'stats_passing_types',
    'gca':'stats_gca',
    'defense':'stats_defense',
    'possession':'stats_possession',
    'playingtime':'stats_playing_time',
    'misc':'stats_misc'
}

In [20]:
collect_tables_in_period("Big5", (2018,2022), section_table_lookup)

	Data saved to datasets/Big5/2017-2018/stats.csv
	Data saved to datasets/Big5/2017-2018/keepers.csv
	Data saved to datasets/Big5/2017-2018/keepersadv.csv
	Data saved to datasets/Big5/2017-2018/shooting.csv
	Data saved to datasets/Big5/2017-2018/passing.csv
	Data saved to datasets/Big5/2017-2018/passing_types.csv
	Data saved to datasets/Big5/2017-2018/gca.csv
	Data saved to datasets/Big5/2017-2018/defense.csv
	Data saved to datasets/Big5/2017-2018/possession.csv
	Data saved to datasets/Big5/2017-2018/playingtime.csv
	Data saved to datasets/Big5/2017-2018/misc.csv
	Data saved to datasets/Big5/2018-2019/stats.csv
	Data saved to datasets/Big5/2018-2019/keepers.csv
	Data saved to datasets/Big5/2018-2019/keepersadv.csv
	Data saved to datasets/Big5/2018-2019/shooting.csv
	Data saved to datasets/Big5/2018-2019/passing.csv
	Data saved to datasets/Big5/2018-2019/passing_types.csv
	Data saved to datasets/Big5/2018-2019/gca.csv
	Data saved to datasets/Big5/2018-2019/defense.csv
	Data saved to data